In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import time

import torch
print(torch.cuda.is_available())  # True 表示可以用 GPU

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
)
cfg.MODEL.DEVICE = "cuda"  # 指定使用 GPU

predictor = DefaultPredictor(cfg)

print("Detectron2 模型運行裝置:", predictor.model.device)

True
Detectron2 模型運行裝置: cuda:0


In [ ]:
# analyze_pitch_from_detectron2.py
# 👉 全自動流程：影片 → Detectron2 → 三動作偵測 → 特徵分析

import os
import json
import time
import shutil
import pandas as pd
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings("ignore")

from pose_utils_detectron2 import load_pose_sequence
from pose_detectron2 import run_detectron2

from detect_release import detect_release
from detect_landing import detect_landing
from detect_shoulder import detect_shoulder

from landing_features import detect_landing_features
from shoulder_features import extract_shoulder_features
from release_features import extract_release_features

# ✅ 單支影片轉特徵 json
def video_2_json(video_path, json_path, description, keypoint_dir, device):
    try:
        if os.path.exists(keypoint_dir):
            # 有舊資料，跳過後續動作
            print(f"{keypoint_dir} 已存在，跳過處理。")
            return 0

        # 🧠 執行 detectron2 輸入影片 → 關節座標資料
        run_detectron2(video_path=video_path, output_dir=keypoint_dir, device=device,save_img=False)
        pose_sequence = load_pose_sequence(keypoint_dir)

        # 🎯 偵測三個關鍵偵
        release_frame = detect_release(pose_sequence)
        landing_frame = detect_landing(pose_sequence, release_frame)
        shoulder_frame = detect_shoulder(pose_sequence, release_frame)

        # 🔍 提取所有特徵
        all_features = {}

        landing_features = detect_landing_features(pose_sequence, landing_frame)
        for key, value in landing_features.items():
            all_features[f'landing_features_{key}'] = value

        shoulder_features = extract_shoulder_features(pose_sequence, shoulder_frame, landing_frame)
        for key, value in shoulder_features.items():
            all_features[f'shoulder_features_{key}'] = value

        release_features = extract_release_features(pose_sequence, release_frame)
        for key, value in release_features.items():
            all_features[f'release_features_{key}'] = value

        all_features['description'] = description

        # 💾 輸出 JSON
        assert ('data' in json_path) and ('features_and_labels' in json_path)
        os.makedirs(os.path.dirname(json_path), exist_ok=True)
        with open(json_path, 'w') as f:
            json.dump(all_features, f, indent=2)
        print('✅ 特徵已儲存至', json_path)

    except Exception as e:
        print('❌ 錯誤發生於影片:', video_path)
        print(e)

ROOT_DIR = 'data'

# ✅ 主程式
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"⚙️ 使用裝置：{device}")

    for subdir in tqdm(os.listdir(ROOT_DIR)):
        subdir_path = os.path.join(ROOT_DIR, subdir)
        if not os.path.isdir(subdir_path):
            print('⚠️ 找不到子目錄')
            continue

        player_name = "_".join(subdir.split("_")[:3])
        csv_path = os.path.join(ROOT_DIR, subdir, f"{player_name}.csv")

        if not os.path.exists(csv_path):
            print(f"⚠️ 找不到對應的 CSV：{csv_path}")
            continue

        df = pd.read_csv(csv_path)

        for video_file in tqdm(os.listdir(subdir_path)):
            if not video_file.endswith('.mp4'):
                continue

            video_path = os.path.join(subdir_path, video_file)

            # ✅ 更穩健的檢查影片對應描述（忽略大小寫）
            row = df[df['Filename'].str.lower() == video_file.lower()]
            if row.empty:
                print(f"⚠️ 找不到影片對應的描述：{video_file}")
                continue

            description = row.iloc[0]['description']

            json_output_path = os.path.join(subdir_path, "features_and_labels", video_file.replace(".mp4", ".json"))
            keypoint_dir = os.path.join(subdir_path, "KEYPOINT_DIR", video_file.replace(".mp4", ""))

            video_2_json(video_path, json_output_path, description, keypoint_dir, device)

if __name__ == "__main__":
    main()


⚙️ 使用裝置：cuda


 13%|█▎        | 27/203 [00:00<00:00, 266.94it/s]

data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0001 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0002 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0003 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0004 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0005 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0006 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0007 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0008 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0009 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0010 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0011 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0012 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0013 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0014 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0015 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_D

  0%|          | 0/238 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1


...

👤 Frame 0: 設定目標人物 ID = 1
✅ Detectron2 處理完畢，骨架資料已儲存。



 35%|███▌      | 72/203 [1:25:54<6:11:12, 170.02s/it]

0 : 33.56817626953125 549.4296875 420.9361877441406
1 : 33.543731689453125 549.5726928710938 421.0738830566406
2 : 33.558074951171875 549.6314697265625 421.0992736816406
3 : 33.568115234375 549.693603515625 421.1211242675781
4 : 33.376922607421875 548.0095825195312 421.6716003417969
5 : 33.549072265625 548.0264892578125 420.6891784667969
6 : 32.627166748046875 548.5643310546875 420.18353271484375
7 : 32.646453857421875 548.4159545898438 421.0362854003906
8 : 33.580780029296875 547.543701171875 421.04193115234375
9 : 33.57684326171875 547.4879150390625 421.0885009765625
10 : 33.596893310546875 547.5425415039062 421.0704345703125
11 : 32.612213134765625 548.3419189453125 420.958740234375
12 : 33.5867919921875 548.2586059570312 420.8219909667969
13 : 33.595123291015625 548.2095947265625 420.7667541503906
14 : 33.58392333984375 548.2813720703125 420.796142578125
15 : 33.59649658203125 547.3598022460938 420.8463439941406
16 : 33.57177734375 547.5756225585938 420.8857421875
17 : 33.573059082

  0%|          | 0/238 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1
